In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 33 (delta 15), reused 26 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 8.20 KiB | 2.73 MiB/s, done.
Resolving deltas: 100% (15/15), done.
/kaggle/working/agnews-classifier


In [35]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=1,
    lora_alpha=3,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query', 'value'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 630,532 || all params: 125,279,240 || trainable%: 0.5033


In [36]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [37]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.371900,1.339291,0.692187
200,1.273300,1.115630,0.892188
300,0.687100,0.365091,0.885938
400,0.333300,0.334964,0.895312
500,0.331600,0.324879,0.898438
600,0.314900,0.330441,0.895312
700,0.325600,0.316865,0.901563
800,0.330500,0.307113,0.909375
900,0.313400,0.314601,0.907813
1000,0.299000,0.312618,0.904687


In [40]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-1600'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [00:58<00:00,  4.27it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-1600.csv


In [39]:
import os
import pandas as pd

# Define paths
csv_path = "/kaggle/working/saved_models/checkpoint-2000/log_history.csv"
output_dir = "/kaggle/working/processed_data"
output_csv = os.path.join(output_dir, "processed_log_history.csv")

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Read the csv file
df = pd.read_csv(csv_path)

# Select desired columns and reorder the dataframe
desired_order = [
    "step",         
    "loss",         
    "eval_loss",   
    "accuracy",     
    "eval_accuracy",
    "learning_rate",
    "epoch"         
]
df = df[desired_order]

# Rename columns for uniformity
df.rename(columns={
    "step": "Step",
    "loss": "Train Loss",
    "eval_loss": "Test Loss",
    "accuracy": "Train Acc",
    "eval_accuracy": "Test Acc",
    "learning_rate": "Learning Rate",
    "epoch": "Epochs"
}, inplace=True)

# Adding loss spread and loss ratio columns
df["loss spread"] = df["Train Loss"] - df["Test Loss"]
df["loss ratio"] = df["Train Loss"] / df["Test Loss"]

# Adding acc spread and acc ratio columns
df["Acc spread"] = df["Test Acc"] - df["Train Acc"]
df["Acc ratio"] = df["Test Acc"] / df["Train Acc"]

# Export the DataFrame as a csv file
df.to_csv(output_csv, index=False)

print(f"Data exported to {output_csv}")

df

Data exported to /kaggle/working/processed_data/processed_log_history.csv


,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,loss spread,loss ratio,Acc spread,Acc ratio
0,100,1.3719,1.339291,0.333125,0.692187,0.000048,0.013405,0.032609,1.024348,0.359062,2.077861
1,200,1.2733,1.115630,0.655357,0.892188,0.000045,0.026810,0.157670,1.141328,0.236830,1.361376
2,300,0.6871,0.365091,0.870536,0.885938,0.000043,0.040214,0.322009,1.881995,0.015402,1.017692
3,400,0.3333,0.334964,0.891964,0.895312,0.000040,0.053619,-0.001664,0.995032,0.003348,1.003754
4,500,0.3316,0.324879,0.887500,0.898438,0.000037,0.067024,0.006721,1.020686,0.010938,1.012324
5,600,0.3149,0.330441,0.893750,0.895312,0.000035,0.080429,-0.015541,0.952969,0.001562,1.001748
6,700,0.3256,0.316865,0.891518,0.901563,0.000033,0.093834,0.008735,1.027568,0.010045,1.011267
7,800,0.3305,0.307113,0.888839,0.909375,0.000030,0.107239,0.023387,1.076150,0.020536,1.023104
8,900,0.3134,0.314601,0.897768,0.907813,0.000028,0.120643,-0.001201,0.996181,0.010045,1.011188
9,1000,0.2990,0.312618,0.903125,0.904687,0.000025,0.134048,-0.013618,0.956440,0.001563,1.001730
